In [ ]:
import pandas as pd
from pathlib import Path
from sad2_final_project.analysis import add_missing_metrics_from_experiment, loader_obsolete_data
import os
from sad2_final_project.analysis import plot_scatter, plot_boxplot, plot_scatter_subplots

# paths
## set global dir
cwd=Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent) 
print(os.getcwd())
## create paths 
DATA_PATH2 = Path('data/trajectory_length_vs_attractors1')

df = loader_obsolete_data(DATA_PATH2 / 'results/metadata.csv', DATA_PATH2 / 'results/joined_results_trajectory_length_vs_attractors.csv')
metrics_list=['TP', 'FP', 'FN', 'precision', 'recall', 'sensitivity', 'AHD', 'SHD', 'EHD', 'SID']
df = add_missing_metrics_from_experiment(df, DATA_PATH2, metrics_list, after_column='attractor_ratio')

# Introduction 
## Goal
The goal of this study is to investigate the relation between **sampling strategy** and **model selection metrics** in *boolean Bayesian networks (BDNs)* using `BNFinder2`. 

In particular, we analyze how characteristics of time-series data generated from Boolean networks influence the accuracy of reconstructing the underlying network structure using Bayesian methods. The experimental factors under investigation include:
- the **trajectory length**,
- the **sampling frequency**, defined as selecting every $n$-th state along a trajectory,
- the ratio between the **number of nodes** and the **trajectory length**, introduced as a normalization parameter $k$.

The generated datasets are grouped into classes determined by:
- the **update mode** (synchronous vs asynchronous),
- the **scoring function** used during inference (MDL and BDe).

In addition, we study **scaling relations with respect to the number of nodes**, aiming to characterize how data requirements and reconstruction accuracy change as network size increases.

# Part 1

## Methodology
### General reasoning 
The primary objective of the experimental design is to isolate how properties of sampled time-series data affect the accuracy of Boolean network reconstruction using `BNFinder2`. 
***
### Update Mode
We distinguish between two fundamentally different update mechanisms:
- **Synchronous update**, which defines a deterministic dynamical system: from any given state, the successor state is uniquely determined.
- **Asynchronous update**, which induces a stochastic process: at each time step, a randomly selected node is updated, leading to multiple possible successor states.
This distinction is critical, as asynchronous dynamics introduce temporal dependence and potential autocorrelation in trajectories. In particular, long residence times in attractors or local cycles may reduce the effective information content of sampled data. Consequently, naive dense sampling may lead to strongly correlated observations, while aggressive subsampling may destroy causal ordering information.
***
### Scoring Functions
We employ two scoring functions implemented in BNFinder2, which differ in how they trade off data fit against model complexity.
Let $G$ denote a candidate network structure and $D$ the observed dataset.

**Minimal Description Length (MDL)**

The MDL score is defined as
$$
\mathrm{MDL}(G \mid D)
= - \log P(D \mid G, \hat{\theta})
\times \frac{1}{2} , |\theta_G| , \log |D|,
$$


where $\hat{\theta}$ are maximum-likelihood parameters and $|\theta_G|$ denotes the number of free parameters implied by the graph structure $G$.

The first term rewards goodness of fit, while the second term penalizes model complexity. As a consequence, MDL favors simpler graphs when data are scarce and becomes less restrictive as sample size increases. This makes MDL particularly sensitive to undersampling and normalization effects.

**Bayesian–Dirichlet equivalence (BDe)**

The BDe score evaluates the marginal likelihood

$$
\mathrm{BDe}(G \mid D)
= \log \int P(D \mid G, \theta), P(\theta \mid G), d\theta,
$$

assuming a Dirichlet prior over conditional probability tables. Under standard assumptions, this integral has a closed-form.

Unlike MDL, BDe incorporates prior beliefs and smooths parameter estimates, which can stabilize inference in low-data regimes but may also reduce sensitivity to subtle structural differences.

By comparing MDL and BDe, we assess whether observed reconstruction effects are driven primarily by data properties or by the inductive bias of the scoring function.

One caveat is that our implementation of those functions is simplified compared to implemented in BNfinder, which may lead to differences in absolute performance. However, relative trends should remain consistent.

---

### Evaluation Metrics
Reconstruction quality is evaluated using structure-based graph distance measures. Each metric captures a distinct notion of discrepancy between the true network $G^\ast$ and the inferred network $\hat{G}$. 

**Adjusted Hamming Distance (AHD)**

Let $A^\ast$ and $\hat{A}$ denote the adjacency matrices of $G^\ast$ and $\hat{G}$. The adjusted Hamming distance is defined as

$$
\mathrm{AHD}(G^\ast, \hat{G})
= \frac{1}{|E^\ast| + |\hat{E}|}
\sum_{i,j} \mathbf{1}_{{A^\ast_{ij} \neq \hat{A}_{ij}}}.
$$
AHD measures the proportion of mismatched edges, normalized by graph size. It penalizes false positives and false negatives symmetrically and allows comparisons across networks of different sizes. This metric serves as the primary measure of structural accuracy.

**Structural Hamming Distance (SHD)**

SHD counts the minimum number of edge insertions, deletions, or reversals required to transform $\hat{G}$ into $G^\ast$.
$
\mathrm{SHD}(G^\ast, \hat{G}) \in \mathbb{N}.
$
While widely used, SHD aggregates heterogeneous error types and does not distinguish between missing, extra, or misoriented edges, limiting its interpretability.

**Structural Intervention Distance (SID)**

SID measures the number of node pairs $(i,j)$ for which the causal effect of intervening on $i$ differs between $G^\ast$ and $\hat{G}$.

$$
\mathrm{SID}(G^\ast, \hat{G})
= \left| {(i,j) : P(j \mid \mathrm{do}(i))*{G^\ast}
\neq P(j \mid \mathrm{do}(i))*{\hat{G}} } \right|.
$$
Where $\mathrm{do}(i=n)$ set node $i$ to have value $n$ at time step, regardles of Boolean update rule.

SID is sensitive to edge orientation and captures errors that affect causal interpretability, even when the undirected structure is largely correct.

Using these metrics jointly allows us to separate purely topological accuracy (AHD, SHD) from correctness of implied causal relationships (SID) and to identify metric-specific effects of sampling and model selection.

---
### Independence
To avoid introducing structural bias, Boolean networks are generated randomly for each condition:
- each node is assigned a random number of parents (uniformly chosen from $\{ 1 ,2 ,3 \}$),
- Boolean update functions are sampled randomly.
All generated networks are accepted without filtering. Repetitions under identical conditions are treated as independent realizations.
### Sample Size Normalization
To ensure comparability across networks of different sizes, we introduce **sample-size normalization**.
Let:
- nnodesn_{\text{nodes}}nnodes​ denote the number of nodes,
- TlengthT_{\text{length}}Tlength​ the trajectory length,
- kkk a normalization constant.
The total number of sampled time points is defined as:
$$
\begin{align}
T_{\text{amount}} & = \frac{{n_{\text{nodes}} \cdot k}}{T_{\text{length}}} 
\end{align}
$$

We fix $k=100$. This choice is motivated by the fact that each node with at most three parents has up to $2^3 = 8$ possible parent-state configurations. Setting $k=100$ corresponds to approximately 10 observations per configuration per node, providing a conservative buffer against stochastic effects and subsampling losses.

### Dataset Construction
#### General Settings
Across all experiments, we vary the following parameters:
* number of nodes (`num_nodes`): ([5, 7, 11, 13, 15]),
* scoring function (`score_function`): MDL, BDe,
* update mode (`update_mode`): synchronous, asynchronous,
* parent count per node: randomly chosen from ({1,2,3}),
* number of repetitions per condition: 30.
---
#### Dataset 1 (Baseline Dataset)
This dataset is used in Experiments 1 and 2.
* trajectory length (`trajectory_length`): ([10, 15, 20, 30, 40, 50]),
* sampling frequency (`sampling_frequency`): ([1, 2, 3, 4, 5]),
* number of trajectories: determined via sample-size normalization.
---
#### Dataset 1.1 (Low-Data Regime)
To probe behavior in extreme data-scarce settings, an auxiliary dataset is constructed with:
* trajectory length: ([5, 7, 9]),
* number of nodes: ([5, 7, 9]),
* sampling frequency: ([1, 2, 3, 4, 5]).
This dataset is included to assess scaling behavior when normalization constraints are tight.
---
#### Dataset 2 (Normalization Study)
Using optimal parameters identified in Experiments 1 and 2, we fix:

$$\begin{align*} \text{sampling frequency} & =\begin{cases}1  & \text{synchronous} \\ 4 & \text{asynchronous}\end{cases} & \mathrm{trajectory\ length}=[0.8 \cdot x]_{\mathrm{round}}\end{align*}$$

We then vary the normalization constant:
$$k \in \{20, 40, 60, 80, 100, 120, 140\}$$

---
### Experiments
#### Experiment 1: Attractor Prevalence and Trajectory Length
Motivated by prior observations that attractor-heavy datasets degrade reconstruction quality, we investigate:
1. the correlation between attractor fraction and reconstruction metrics,
2. how attractor prevalence depends on trajectory length,
3. how these effects scale with network size.
We introduce the **scale ratio**
$$\text{scale ratio} = \frac{T_{\text{length}}}{n_{\text{nodes}}}$$
and analyze its relationship with attractor fraction to identify regimes that balance coverage of transient dynamics and attractor exploration.

---
#### Experiment 2: Subsampling and Temporal Dependence
This experiment examines whether there exists relation between ESS and metrics score functions subsampling improves reconstruction quality by reducing temporal dependence.
We first analyze:
* **effective sample size (ESS)**,
* **autocorrelation factor (ACF)**,
as functions of update mode and sampling frequency.
We then assess monotonic relationships between ESS and reconstruction metrics using Spearman correlation. Statistical significance of performance differences between sampling frequencies is evaluated using paired Wilcoxon tests, applied only when sufficient paired observations are available.
To avoid confounding effects scale ratios below 1.5 are excluded and cases without effective subsampling are removed.
---
#### Experiment 3: Normalization and Information Saturation
Fixing sampling parameters based on earlier results, we analyze how reconstruction metrics evolve as a function of the normalization constant (k). This allows us to identify saturation regimes in which increasing sample size yields diminishing returns.

---
#### Experiment 4: Score Function Sensitivity
Finally, using a subset of Experiment 3 with fixed normalization, we investigate how reconstruction metrics depend on the choice of scoring function. The goal is to understand whether differences in reconstruction quality arise from data properties or from intrinsic characteristics of the scoring criteria.

## Analysis

### Experiment 1 

### Experiment 2

### Experiment 3

### Experiment 4

# Part 2

## Methodology

## Analysis